In [1]:
import numpy as np 
import pandas as pd 
import spacy 

from pickle import dump, load 
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense, GRU, Dropout
from keras.utils import to_categorical
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [16]:
nlp = spacy.load('en_core_web_md', disable=['parser', 'ner', 'tagger'])
nlp.max_length = 5_000_000

In [17]:
# Read each line and store as a list
with open('first_1000_lines.txt', 'r', encoding='utf-8') as file:
    lines = file.readlines()

# Strip newline characters and put into a DataFrame
data = pd.DataFrame([line.strip() for line in lines], columns=['line'])
print(data.head())

                                                line
0                                            "ACT I"
1                     "SCENE I. London. The palace."
2  "Enter KING HENRY, LORD JOHN OF LANCASTER, the...
3           "So shaken as we are, so wan with care,"
4       "Find we a time for frighted peace to pant,"


In [18]:
data.dropna(inplace=True)
data.shape 

(1000, 1)

In [31]:
data = data[data['line'].str.len() > 2]
data.shape

(1000, 1)

In [35]:
def is_valid_line(line):
    line = line.strip().lower()
    
    # Remove surrounding double quotes
    if line.startswith('"') and line.endswith('"'):
        line = line[1:-1]

    # Remove empty lines
    if len(line) < 3:
        return False
    
    # Remove known stage directions
    if re.match(r'^(scene|act|enter|exeunt|exit|manet)', line):
        return False

    # Remove lines in brackets or all uppercase (common for directions)
    if line.startswith('[') or line.startswith('('):
        return False
    if line.upper() == line and len(line.split()) < 5:
        return False

    return True

In [36]:
clean_lines = [line.strip('"') for line in data['line'] if is_valid_line(line)]
print(clean_lines[:10])

['So shaken as we are, so wan with care,', 'Find we a time for frighted peace to pant,', 'And breathe short-winded accents of new broils', 'To be commenced in strands afar remote.', 'No more the thirsty entrance of this soil', "Shall daub her lips with her own children's blood,", 'Nor more shall trenching war channel her fields,', 'Nor bruise her flowerets with the armed hoofs', 'Of hostile paces: those opposed eyes,', 'Which, like the meteors of a troubled heaven,']
